In [1]:
import torch
import torch.nn as nn

In [7]:
a = torch.rand(2, 1)
b = torch.zeros(2)
print(a, b)
nn.BCEWithLogitsLoss(a, b)

RuntimeError: Boolean value of Tensor with more than one value is ambiguous